# Exploratory data analysis with Pandas - dataset-wide analysis of per-series unique values

In [ ]:
hpc_predict_data_dir="/home/lukasd/src/hpc-predict/data/v0" # e.g. ../../../data/v0

This script assumes that you have the Freiburg dataset available, otherwise first run `data/fetch_scripts/fetch_freiburg.sh`.

To run the following steps, the DICOM headers must previously be converted to pandas tables, i.e.

```
python convert_dicom_to_pandas.py --mri-data-root "${hpc_predict_data_dir}/input_data/original/mri/MRT Daten Bern" --mri-samples ... --output-root "${hpc_predict_data_dir}/input_data/preprocessed/mri/MRT Daten Bern DICOM Header"
```

Then extract per DICOM series unique values by running `extract_unique_tags_pandas.py` on serialized pandas `DataFrame`s:

```
python extract_unique_tags_pandas.py --headers-root "${hpc_predict_data_dir}/input_data/preprocessed/mri/MRT Daten Bern DICOM Header" --mri-samples ... --output-root "${hpc_predict_data_dir}/input_data/preprocessed/mri/MRT Daten Bern DICOM Header Unique"
```

## Parse extracted unique DICOM header lists

In [ ]:
import os
from glob import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import pydicom

import matplotlib.dates as mdates
import re

pkls = sorted(glob(hpc_predict_data_dir + "/input_data/preprocessed/mri/MRT Daten Bern DICOM Header Unique/*.pkl"), key=lambda x: int(os.path.basename(x).split('.')[0]) )
pkls

In [ ]:
pd.set_option('display.max_columns', None)

df_unique = []
df_skipped = []
for pkl in pkls:
    read_pkl = pd.read_pickle(pkl)
    labels, counts = np.unique(read_pkl.columns.values, return_counts=True)
    print("Dropping {} from {}".format(labels[counts > 1], os.path.basename(pkl)))
    read_pkl = read_pkl.drop(labels=labels[counts > 1], axis=1)

    read_pkl_deidentified = read_pkl['De-identification Method Code Sequence'].apply(lambda x: hasattr(x,'__len__') and len(x) == 1 and hasattr(x[0],'__len__') and len(x[0]) > 0)
    if (~read_pkl_deidentified).sum() > 0:
        print("WARNING: Found {} series in studies {} in {} without de-identification tags - skipping them!".format(read_pkl_deidentified.sum(), list(read_pkl[~read_pkl_deidentified]['Study Instance UID'].apply(lambda x: x[0]).unique()), os.path.basename(pkl)))
        df_skipped.append(read_pkl[~read_pkl_deidentified])
    read_pkl = read_pkl[read_pkl_deidentified]
    
    df_unique.append(read_pkl)

df_unique = pd.concat(df_unique, axis=0)
if len(df_skipped) > 0:
    df_skipped = pd.concat(df_skipped, axis=0) 
else:
    df_skipped = None
df_unique

### Validate anonymization with de-identification tags

In [ ]:
if not df_skipped is None:
    print("Checking skipped data for any non-anonymized DICOM images")
    df_skipped_folders = {collection: pd.read_pickle("/home/lukasd/src/hpc-predict/data/v0/input_data/preprocessed/mri/MRT Daten Bern DICOM Header/{}.pkl".format(collection)) for collection in df_skipped['FileCollectionID'].apply(lambda x: x[0]).unique()}
    for collection, df_skipped_folder in df_skipped_folders.items():
        print("Evaluating FileCollectionID {} for missing de-identification tags".format(collection))

        df_skipped_folder_hashified = df_skipped_folder.rename(columns=df_skipped_folder.apply( lambda c: c.dropna().apply(lambda x: x.name).unique(), axis=0).iloc[0].to_dict())\
                                                       .applymap(lambda x: unpack_pydicom_value(x.value) if pd.notnull(x) else x)
        df_skipped_folder_anonymized = df_skipped_folder_hashified['De-identification Method Code Sequence'].apply(lambda x: hasattr(x,'__len__') and len(x) > 0)
        df_skipped_folder_unaffected = df_skipped_folder_hashified[df_skipped_folder_anonymized]
        df_skipped_folder_affected = df_skipped_folder_hashified[~df_skipped_folder_anonymized]

        print("Affected studies:   {} ".format(df_skipped_folder_affected['Study Instance UID'].unique()))
        print("      -> folders:   {} ".format(df_skipped_folder_affected['FilePath'].apply(lambda x: '/'.join(x.split('/')[-5:-2]) ).unique()))
        print("Unaffected studies: {} ".format(df_skipped_folder_unaffected['Study Instance UID'].unique()))
        print("      ->   folders: {} ".format(df_skipped_folder_unaffected['FilePath'].apply(lambda x: '/'.join(x.split('/')[-5:-2]) ).unique()))
else:
    print("No skipped data - no DICOM anonymization issues w.r.t. \'De-identification Method Code Sequence\'-tag found.")

## Overview of sequences

In [ ]:
plt.rcParams.update({'font.size': 14})
plt.hist(df_unique['Sequence Name'].apply(lambda x: repr(tuple(x))).values)
plt.title('Sequence Name')
plt.show()

In [ ]:
# df_unique[df_unique['Sequence Name'] == 'fl3d1']

In [ ]:
# Inspect the null-values (e.g. for debugging)

# # Full dataset
# df_unique.applymap(lambda x: len(x) if np.all(pd.notnull(x)) else x)

# # Sequence-wise
# df_unique[df_unique['Sequence Name'] == 'Nav'].applymap(lambda x: len(x) if np.all(pd.notnull(x)) else x)

# # Columns with null-values (sequence-name-agnostic!)
# df_unique.columns[df_unique.applymap(lambda x: np.isnan(x) if not isinstance(x,np.ndarray) else False).any()]

## Sequence-specific analysis: choose sequence to analyze

In [ ]:
analyzed_sequence = 'fl3d1'
df_unique_seq = df_unique[df_unique['Sequence Name'].apply(lambda x: x[0]) == analyzed_sequence]

# # Truncate columns for binary search of pathological column 
# df_unique_seq = df_unique_seq.drop(np.concatenate([df_unique_seq.columns.values[:76*len(df_unique_seq.columns.values)//128],df_unique_seq.columns.values[77*len(df_unique_seq.columns.values)//128:]]), axis=1)

### Here you can add columns of interest such as Trigger Time interval divided by heart cycle period (Nominal Interval) that will be plotted

In [ ]:
def trigger_time_nominal_interval_ratio(row):
    trigger_time = row['Trigger Time']
    return (trigger_time.max()-trigger_time.min())/row['Nominal Interval'].mean()
    
if analyzed_sequence == 'fl3d1':
    df_unique_seq = pd.concat([df_unique_seq, pd.DataFrame({'Derived: Trigger Time range/Nominal Interval': df_unique_seq.apply(lambda x: np.array([trigger_time_nominal_interval_ratio(x)]),axis=1),
                                                            'Derived: Trigger Time Min': df_unique_seq.apply(lambda x: np.array([x['Trigger Time'].min()]),axis=1),
                                                            'Derived: Nominal Interval - Trigger Time Max': df_unique_seq.apply(lambda x: np.array([x['Nominal Interval'].mean() - x['Trigger Time'].max()]),axis=1),
                                                           })], axis=1)

### Display columns that will be discarded

In [ ]:
# Visualize discarded columns computed from different conditions (can be altered) via set differences
if analyzed_sequence == 'Nav':
#     set_diff = set(df_unique_seq.columns[df_unique_seq.apply(lambda x: x.apply(lambda y: (isinstance(y,float) and np.isnan(y)) or (len(y) > 0 and isinstance(y[0],bytes)) or ( len(y) > 3 ) ), axis=1).any()].values) - \
#                set(df_unique_seq.columns[df_unique_seq.apply(lambda x: x.apply(lambda y: (isinstance(y,float) and np.isnan(y)) or ( len(y) > 3 ) ), axis=1).any()].values)
    set_diff = set(df_unique_seq.columns[df_unique_seq.apply(lambda x: x.apply(lambda y: (hasattr(y,'__len__') and len(y) > 5 ) ), axis=1).any()].values) - \
               set(df_unique_seq.columns[df_unique_seq.apply(lambda x: x.apply(lambda y: False ), axis=1).any()].values)
else:
#     set_diff = set(df_unique_seq.columns[df_unique_seq.apply(lambda x: x.apply(lambda y: (isinstance(y,float) and np.isnan(y)) or (len(y) > 0 and isinstance(y[0],bytes)) or ( len(y) >= np.min([*x['Cardiac Number of Images'],len(x['Slice Location'])]) ) ), axis=1).any()].values) - \
#                set(df_unique_seq.columns[df_unique_seq.apply(lambda x: x.apply(lambda y: (isinstance(y,float) and np.isnan(y)) or ( len(y) >= np.min([*x['Cardiac Number of Images'],len(x['Slice Location'])]) ) ), axis=1).any()].values)
    set_diff = set(df_unique_seq.columns[df_unique_seq.apply(lambda x: x.apply(lambda y: (hasattr(y,'__len__') and len(y) >= np.min([*x['Cardiac Number of Images'],len(x['Slice Location'])]) ) ), axis=1).any()].values) - \
               set(df_unique_seq.columns[df_unique_seq.apply(lambda x: x.apply(lambda y: False ), axis=1).any()].values)
df_unique_seq[ [el for el in set_diff] ]

In [ ]:
# Discard the above visualized columns
if analyzed_sequence == 'Nav':
    # Eliminate per-image varying information, bytes
#     df_unique_drop_columns = df_unique_seq.columns[df_unique_seq.apply(lambda x: x.apply(lambda y: (isinstance(y,float) and np.isnan(y)) or (len(y) > 0 and isinstance(y[0],bytes)) or ( len(y) > 3 ) ), axis=1).any()]
    df_unique_too_long_columns = df_unique_seq.columns[df_unique_seq.apply(lambda x: x.apply(lambda y: (hasattr(y,'__len__') and len(y) > 5 ) ), axis=1).any()]
else:
    # Eliminate spatio-temporal information
#     df_unique_drop_columns = df_unique_seq.columns[df_unique_seq.apply(lambda x: x.apply(lambda y: (isinstance(y,float) and np.isnan(y)) or (len(y) > 0 and isinstance(y[0],bytes)) or ( len(y) >= np.min([*x['Cardiac Number of Images'],len(x['Slice Location'])]) )  ), axis=1).any()]
    df_unique_too_long_columns = df_unique_seq.columns[df_unique_seq.apply(lambda x: x.apply(lambda y: (hasattr(y,'__len__') and len(y) >= np.min([*x['Cardiac Number of Images'],len(x['Slice Location'])]) )  ), axis=1).any()]
print("Dropping too long (spatio-temporal information in fl3d1) columns: {}".format(df_unique_too_long_columns))
df_unique_seq_sanitized = df_unique_seq.drop(df_unique_too_long_columns, axis=1)

# Choose sequence to visualize
df_unique_all_nan_empty_bytes_columns = df_unique_seq_sanitized.columns[df_unique_seq_sanitized.applymap(lambda x: (isinstance(x,float) and np.isnan(x)) or (hasattr(x,'__len__') and len(x) == 0) or (len(x) > 0 and isinstance(x[0],bytes)) ).all()]
print("Dropping all-NaN/empty/bytes columns: {}".format(df_unique_all_nan_empty_bytes_columns))
df_unique_plot = df_unique_seq_sanitized.drop(df_unique_all_nan_empty_bytes_columns, axis=1)
# df_unique_plot = df_unique_seq_sanitized.drop(df_unique_seq_sanitized.columns[df_unique_seq_sanitized.applymap(lambda x: len(x) == 0).all()], axis=1)


### Perform visualization (incl. some final type conversions)

In [ ]:
plt.rcParams.update({'font.size': 18})

# Show non-unique tags
print("Columns with multiple values (treated as stacked): {}.".format(df_unique_plot.columns[df_unique_plot.applymap(lambda x: hasattr(x,'__len__') and len(x) > 1).any()]))

# # df_unique_plot_repr = df_unique_plot.apply(lambda x: x.apply(lambda y: y[0]) if x.apply(lambda y: len(y) == 1).all() else x.apply(lambda y: tuple(y)) )
# tuple_with_newlines = lambda x : x if not isinstance(x,tuple) else '(' + ',\n'.join([str(el) for el in x]) + ')'
# df_unique_plot_repr = df_unique_plot.apply(lambda x: x.apply(lambda y: tuple_with_newlines(y[0])) if x.apply(lambda y: len(y) == 1).all() else x.apply(lambda y: tuple([tuple_with_newlines(el) for el in y])) )

# num_columns = len(df_unique_plot_repr.columns)
num_columns = len(df_unique_plot.columns)
ncols=4
nrows=(num_columns+ncols-1)//ncols
fig, ax = plt.subplots(nrows=nrows , ncols=ncols, figsize=(12*ncols,20*nrows))
axs = ax.flatten()

# for i, col in enumerate(df_unique_plot_repr.columns[:num_columns]):
for i, col in enumerate(df_unique_plot.columns[:num_columns]):
    
    # Filter any left over NaNs
    df_unique_plot_is_not_nan = df_unique_plot[col].apply(lambda y: not( (isinstance(y,float) and np.isnan(y)) ) )
    df_unique_plot_col = df_unique_plot[col][df_unique_plot_is_not_nan]
    
    def get_tuple_type(el):
        if not isinstance(el,(tuple,list,np.ndarray)):
            return type(el)
        else:
            return tuple([get_tuple_type(el_i) for el_i in el])
            
#     value_types = df_unique_plot_repr[col].apply(type).values
    value_types = set()
    for t in df_unique_plot_col.apply(lambda x: get_tuple_type(x) ).unique(): # unique types across series
        # print("Value type: {}".format(t), flush=True)
        for t_i in t: # unique types within series
            value_types.add(t_i)
    print("Plotting {}".format(col))
    print("    value types: {}".format(value_types), flush=True)
    if len(value_types) > 1:
        print("WARNING: column {} with heterogeneous value types: {}".format(col, value_types), flush=True)

    def get_plot_value(col, series_value, value_types): # TODO: For UID values get only the prefix - like '.'.join(uid.split('.')[:-1]) ############################
        if isinstance(series_value,tuple):
            return  '(' + ',\n'.join([str(el) for el in series_value]) + ')'
        elif col == 'Patient\'s Age':
            assert isinstance(series_value, str)
            return int( re.search(r'^(?P<age>\d{3})Y$', series_value).group('age') )
        elif col == 'Patient\'s Sex':
            return series_value if series_value != 'W' else 'F'
        elif isinstance(series_value, datetime.date):
            if len(value_types) == 1:
                return mdates.date2num(series_value)
            else: # str is fallback representation
                return str(series_value)                
        elif isinstance(series_value, datetime.time):
            if len(value_types) == 1:
                return mdates.date2num(datetime.datetime(1900,1,1,
                                                     series_value.hour, series_value.minute, series_value.second, 
                                                     series_value.microsecond, series_value.tzinfo))
            else: # str is fallback representation
                return str(series_value)                
        elif isinstance(series_value, pydicom.valuerep.PersonName): # or col == 'Rescale Type':
            return str(series_value)
        elif isinstance(series_value, str) or np.isscalar(series_value):
            return series_value
        else:
            raise ValueError("No treatment for {} of type {}".format(series_value, type(series_value)))

    # Distinguish all single-valued from multi-valued columns
    if df_unique_plot_col.apply(lambda y: len(y) == 1).all():
        df_unique_plot_col_finished = df_unique_plot_col.apply(lambda x: get_plot_value(col, x[0], value_types)) # newlines are rather for tick labels
    else:
        df_unique_plot_col_finished = df_unique_plot_col.apply(lambda x: tuple([ get_plot_value(col, x_i, value_types) for x_i in x])) # newlines are rather for tick labels
        if df_unique_plot_col_finished.apply(lambda y: len(y) == 0).any(): # e.g. needed if col == 'Rescale Type':
            df_unique_plot_col_finished = df_unique_plot_col_finished.apply(repr)

    axs[i].hist(df_unique_plot_col_finished.values, label=col, bins=30, stacked=True)
    
    if datetime.date in value_types and len(value_types) == 1:
#         axs[i].xaxis.set_major_locator(mdates.YearLocator())
#         axs[i].xaxis.set_major_formatter(mdates.DateFormatter('%d.%m.%y'))
        locator = mdates.AutoDateLocator()
        axs[i].xaxis.set_major_locator(locator)
        axs[i].xaxis.set_major_formatter(mdates.AutoDateFormatter(locator))
    elif datetime.time in value_types and len(value_types) == 1:
        axs[i].xaxis.set_major_locator(mdates.HourLocator())
        axs[i].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
        
    
#     if col != 'Rescale Type':
#         axs[i].hist(df_unique_plot_col_finished.values, label=col, bins=30, stacked=True)
#     else:
#         axs[i].hist(df_unique_plot_col_finished.apply(repr).values, label=col, bins=30, stacked=True)
        

    
#     if len(value_types) == 1:
#         if isinstance(list(value_types)[0],tuple): # maybe this can also handle situations with multiple values per series, each being a tuple
#             axs[i].hist(df_unique_plot_col.apply(lambda y: '(' + ',\n'.join([str(el) for el in y[0]]) + ')').values , label=col, bins=30, stacked=True)            
#         elif not(datetime.time in value_types or datetime.date in value_types or pydicom.valuerep.PersonName in value_types or col == 'Rescale Type' ):
#             axs[i].hist(df_unique_plot_col.values, label=col, bins=30, stacked=True)
#         else:
#             axs[i].hist(df_unique_plot_col.apply(str).values, label=col, bins=30, stacked=True)
#     else:
#         print("Column {} with heterogeneous value types: {}".format(col, value_types), flush=True)
#         axs[i].hist(df_unique_plot_col.values, label=col, bins=30, stacked=True)
        

#     if not(datetime.time in value_types or datetime.date in value_types or pydicom.valuerep.PersonName in value_types or col == 'Rescale Type' ):
#         axs[i].hist(df_unique_plot_repr[col].values, label=col, bins=30, stacked=True)
#     else:
#         axs[i].hist(df_unique_plot_repr[col].apply(str).values, label=col, bins=30, stacked=True)
# #         axs[i].hist(df_unique_plot_repr.apply(lambda x: datetime.datetime(x[col[:-4]+"Date"].year, x[col[:-4]+"Date"].month, x[col[:-4]+"Date"].day, x[col].hour, x[col].minute, x[col].second, x[col].microsecond), axis=1).values, label=col, bins=30, stacked=True)        

    title_num_nans = (~df_unique_plot_is_not_nan).sum()
    title_num_rows = df_unique_plot_is_not_nan.shape[0]
    title_addendum = " ({}/{} NaNs excluded)".format(title_num_nans, title_num_rows) if title_num_nans != 0 else ""
    axs[i].set_title(col + title_addendum) # could also add value_types if desired
    axs[i].tick_params(labelrotation=90)

fig.tight_layout()